# 自适应风格迁移效果展示

In [ ]:
# 导入代码
import os
import numpy as np
import matplotlib.pyplot as plt #画图
from PIL import Image #读取图片
import torch.nn as nn 
import torch
from torchvision import transforms #变换
from torchvision.utils import save_image #把torch输出存到图片

这里为了便于展示，所以采用的是kaggle的一个比较小的艺术画数据集，和原本训练的数据集不是同一个，但是也可以应用。

In [ ]:
imgs = []
for dirname, _, filenames in os.walk('/kaggle/input/best-artworks-of-all-time/resized'):
    #这个是数据集目录
    for filename in filenames:
        x = os.path.join(dirname, filename)
        if x.endswith('.jpg'):
            imgs.append(x)#遍历记录当前有的图片
def random_select(seed=2333):
    np.random.seed(seed)#随机数种子
    content = np.random.choice(imgs)
    style = np.random.choice(imgs)
    return content,style#返回选取的结果

In [ ]:
content,style = random_select(114514)#挑一组内容-风格，都是画作

In [ ]:
c = Image.open(content)
s =  Image.open(style)#读取
plt.subplot(1,2,1)
plt.title('Content')
plt.imshow(c)
plt.subplot(1,2,2)
plt.title('Style')
plt.imshow(s)
plt.tight_layout()#展现

这个是数据预处理的函数部分

In [ ]:
def test_transform(size, crop):#封装了需要用到的数据预处理
    transform_list = []
    if size != 0:#输入0就不进行resize，否则进行resize，把短边调整到size
        transform_list.append(transforms.Resize(size))
    if crop:#选择进行CenterCrop操作，会切成size*size大小
        transform_list.append(transforms.CenterCrop(size))
    transform_list.append(transforms.ToTensor())#处理最后一步是转换到tensor，因为现在是数组
    transform = transforms.Compose(transform_list)#构成处理序列
    return transform

这个是风格迁移的实行函数，需要用到两个模型，已经附在kaggle notebook内。

In [ ]:
def style_transfer(vgg, decoder, content, style, alpha=1.0,
                   interpolation_weights=None):
    assert (0.0 <= alpha <= 1.0)#风格渲染这里的参数要保证输出是原图和风格的凸组合，所以在0-1
    content_f = vgg(content)
    style_f = vgg(style)#vgg提取信息
    if interpolation_weights:#如果有风格插值，Style输入是多维的
        _, C, H, W = content_f.size()
        feat = torch.FloatTensor(1, C, H, W).zero_().to(device)
        base_feat = adaptive_instance_normalization(content_f, style_f)
        for i, w in enumerate(interpolation_weights):
            feat = feat + w * base_feat[i:i + 1]#累加多种风格的作用
        content_f = content_f[0:1]#一个batch的风格是一样的，只取一个
        #这个是因为如果用风格插值模式，实际操作是复制k组内容和k个不同风格做迁移，然后合并
    else:
        feat = adaptive_instance_normalization(content_f, style_f)#没有插值的话直接处理
    feat = feat * alpha + content_f * (1 - alpha)#风格渲染信息与原本内容叠加的凸组合
    return decoder(feat)#返回从混合数据解码的图片信息

导入模型和pytorch定义好的函数：

In [ ]:
import sys
sys.path.append('../usr/lib/adain-model-define')#依赖的模型定义文件
from adain_model_define import *

decoder_path = '../input/adainmodel/decoder_iter_200000.pth'#存好的模型
vgg_path = '../input/adainmodel/vgg_normalised.pth'
device = 'cpu'#其实kaggle也可以GPU，但是这里CPU也很快

decoder.eval()
vgg.eval()#调整到测试模式

decoder.load_state_dict(torch.load(decoder_path))
vgg.load_state_dict(torch.load(vgg_path))#加载数据
vgg = nn.Sequential(*list(vgg.children())[:31])#VGG只用前面部分，即0-31，前32层

vgg.to(device)
decoder.to(device)#调整模型位置，这里就直接CPU

content_size = 512
style_size = 512 #图片大小
crop = False #如果为真，做中心裁剪


content_tf = test_transform(content_size, crop)
style_tf = test_transform(style_size, crop)# Transformation对象


In [ ]:
c_trans = content_tf(c)
plt.imshow(c_trans[0,:,:])
#pytorch特色，C在第一个维度，所以看一下大概情况，只取一个channel可视化

In [ ]:
alpha = 1#风格强度
preserve_color = False #保存颜色，如果选这个，要在输出前先把风格图的颜色信息拉到内容图的RGB均值
c_trans = content_tf(c)
s_trans = style_tf(s)#预处理变换
if preserve_color:
    s_trans = coral(s_trans,c_trans)#颜色调整
c_trans = c_trans.to(device).unsqueeze(0)
s_trans = s_trans.to(device).unsqueeze(0)#补出batch=1的第一个维度来
with torch.no_grad():#不记录梯度的模式
    output = style_transfer(vgg, decoder, c_trans, s_trans,
                            alpha)#运行
    output = output.cpu()#调到CPU（其实如果现在device就是CPU，并不用）
output_name = './stylized.png'
save_image(output, output_name)#用torch自带函数提供导出

此处已经导出并存储，下面进行展示，更换开始的随机数种子可以尝试不同的图片进行迁移的效果。

In [ ]:
plt.subplot(1,3,1)
plt.title('Content')
plt.imshow(c)
plt.subplot(1,3,2)
plt.title('Style')
plt.imshow(s)
plt.subplot(1,3,3)
plt.title('Transfer')
plt.imshow(Image.open('./stylized.png'))
plt.tight_layout()#看一下效果